**This program will parse your data of midi files into data readable by the ai**

Import

In [1]:
!pip install py_midicsv
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.5 MB/s eta 0:00:00


In [2]:
!mkdir /content/musicFixed/

**(Upload midi now to content folder) Convert the midi into all one track**

In [4]:
import os
import mido

def merge_midi_tracks(input_file):
    try:
      mid = mido.MidiFile("/content/" + input_file)

      merged = mido.MidiFile(ticks_per_beat=mid.ticks_per_beat)

      # Create a new track to merge the MIDI events
      merged_track = mido.MidiTrack()
      merged.tracks.append(mido.merge_tracks(mid.tracks))

      # Save the merged MIDI file

      merged.save("/content/musicFixed/" + input_file)
      print(input_file)
    except Exception as e:
      print(e)
      pass

for filename in os.listdir("/content/"):
    if filename.endswith('.mid'):
      merge_midi_tracks(filename)

Liar (Ben weber horoitz.mid


In [5]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/musicFixed/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  outValue = str(note)

  if(instrument<=127):
    outValue += "%"

  outValue += str(start)

  if(velocity > 108):
    outValue += "$"
  elif velocity > 74:
    outValue += "#"
  elif velocity > 40:
    outValue += "@"
  else:
    outValue += "!"

  outValue += str(length)

  return outValue

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
for midi_file in midi_files:
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    csv_parsed = []
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

for i in range(0, len(csv_string), 1800):
  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)